In [1]:
# Importa a biblioteca duckdb para manipular o banco de dados
import duckdb

# Cria a conexão com o banco de dados 'dados_duckdb.db'
con = duckdb.connect(database='dados_duckdb.db', read_only=False)

# Executa uma consulta para visualizar todos os dados da tabela bronze_z0019
con.execute("SELECT * FROM bronze_z0019").df()


,NATBR,MATKT,WERKS,MAINS,LABST,nome_arquivo,data_ingestao
0,10001,PARAFUSO,BT10,100,100,z0019_1.csv,2025-08-08 06:24:36.622840
1,10002,MARTELO,BT56,100,1500,z0019_1.csv,2025-08-08 06:24:36.622840
2,10003,PREGO,BT10,100,50,z0019_1.csv,2025-08-08 06:24:36.622840
3,10004,SERRA,BT56,100,200,z0019_2.csv,2025-08-08 06:43:33.813065
4,10005,MACHADO,BT56,100,100,z0019_2.csv,2025-08-08 06:43:33.813065
5,10003,PREGO,BT10,100,60,z0019_2.csv,2025-08-08 06:43:33.813065


In [3]:
#ou assim tambem
# Executa uma consulta SQL para selecionar todos os registros da tabela bronze_z0019
# e armazena o resultado em um DataFrame chamado 'df'
df = con.execute("SELECT * FROM bronze_z0019").fetchdf()

# Exibe as 10 primeiras linhas do DataFrame para visualização inicial, mas neste caso so tenho 6 registros, do sero ao 5
df.head(10)


,NATBR,MATKT,WERKS,MAINS,LABST,nome_arquivo,data_ingestao
0,10001,PARAFUSO,BT10,100,100,z0019_1.csv,2025-08-08 06:24:36.622840
1,10002,MARTELO,BT56,100,1500,z0019_1.csv,2025-08-08 06:24:36.622840
2,10003,PREGO,BT10,100,50,z0019_1.csv,2025-08-08 06:24:36.622840
3,10004,SERRA,BT56,100,200,z0019_2.csv,2025-08-08 06:43:33.813065
4,10005,MACHADO,BT56,100,100,z0019_2.csv,2025-08-08 06:43:33.813065
5,10003,PREGO,BT10,100,60,z0019_2.csv,2025-08-08 06:43:33.813065


In [4]:
#ordenar pela data de injestao, produto prego oultimo registro e mais atualizada e o valor e maior

# Cria um DataFrame com os dados da tabela bronze_z0019,
# filtrando apenas os registros com data de ingestão maior ou igual a 2025-01-11 neste caso todos de hje 

df = con.execute("""
    SELECT *
    FROM bronze_z0019
    WHERE data_ingestao >= '2025-01-11'
""").fetchdf()

# Visualiza os 10 primeiros registros do resultado filtrado
df.head(10)


,NATBR,MATKT,WERKS,MAINS,LABST,nome_arquivo,data_ingestao
0,10001,PARAFUSO,BT10,100,100,z0019_1.csv,2025-08-08 06:24:36.622840
1,10002,MARTELO,BT56,100,1500,z0019_1.csv,2025-08-08 06:24:36.622840
2,10003,PREGO,BT10,100,50,z0019_1.csv,2025-08-08 06:24:36.622840
3,10004,SERRA,BT56,100,200,z0019_2.csv,2025-08-08 06:43:33.813065
4,10005,MACHADO,BT56,100,100,z0019_2.csv,2025-08-08 06:43:33.813065
5,10003,PREGO,BT10,100,60,z0019_2.csv,2025-08-08 06:43:33.813065


In [5]:
# Cria uma variável chamada df que irá armazenar os dados da tabela bronze_z0019
# Mas antes, aplicamos uma lógica para remover duplicatas com base no campo NATBR

df = con.execute("""
    SELECT * 
    FROM (
        -- Aplica uma numeração por linha para cada NATBR, ordenando pela data de ingestão mais recente
        SELECT *,
               ROW_NUMBER() OVER (
                   PARTITION BY NATBR
                   ORDER BY data_ingestao DESC
               ) AS row
        FROM bronze_z0019
    )
    -- Seleciona apenas os registros mais recentes de cada NATBR
    WHERE row = 1
""").fetchdf()

# Exibe os 10 primeiros registros resultantes
df.head(10)


,NATBR,MATKT,WERKS,MAINS,LABST,nome_arquivo,data_ingestao,row
0,10003,PREGO,BT10,100,60,z0019_2.csv,2025-08-08 06:43:33.813065,1
1,10004,SERRA,BT56,100,200,z0019_2.csv,2025-08-08 06:43:33.813065,1
2,10001,PARAFUSO,BT10,100,100,z0019_1.csv,2025-08-08 06:24:36.622840,1
3,10005,MACHADO,BT56,100,100,z0019_2.csv,2025-08-08 06:43:33.813065,1
4,10002,MARTELO,BT56,100,1500,z0019_1.csv,2025-08-08 06:24:36.622840,1


In [6]:
# vai adicionar um incremental no id, ordem decrescente(desc) na data de injestao agrupado por NATBR
# registros mais atualizados tem 
# neste caso o registro2 do produto prego e o mais atualizado
df = con.execute("""
    SELECT *, ROW_NUMBER() OVER (PARTITION BY NATBR ORDER BY data_ingestao DESC) AS row
    FROM bronze_z0019
    WHERE data_ingestao >= '2025-01-11'
""").fetchdf()
df.head(10)

,NATBR,MATKT,WERKS,MAINS,LABST,nome_arquivo,data_ingestao,row
0,10004,SERRA,BT56,100,200,z0019_2.csv,2025-08-08 06:43:33.813065,1
1,10002,MARTELO,BT56,100,1500,z0019_1.csv,2025-08-08 06:24:36.622840,1
2,10003,PREGO,BT10,100,60,z0019_2.csv,2025-08-08 06:43:33.813065,1
3,10003,PREGO,BT10,100,50,z0019_1.csv,2025-08-08 06:24:36.622840,2
4,10005,MACHADO,BT56,100,100,z0019_2.csv,2025-08-08 06:43:33.813065,1
5,10001,PARAFUSO,BT10,100,100,z0019_1.csv,2025-08-08 06:24:36.622840,1


In [7]:
# seleciona so as linhas = a 1, agora apenas 5 registros
df = con.execute("""
    SELECT *
    FROM (
        SELECT *, ROW_NUMBER() OVER (PARTITION BY NATBR ORDER BY data_ingestao DESC) AS row
        FROM bronze_z0019
        WHERE data_ingestao >= '2025-01-11'
    ) 
    WHERE row = 1
""").fetchdf()

df.head(10)


,NATBR,MATKT,WERKS,MAINS,LABST,nome_arquivo,data_ingestao,row
0,10003,PREGO,BT10,100,60,z0019_2.csv,2025-08-08 06:43:33.813065,1
1,10005,MACHADO,BT56,100,100,z0019_2.csv,2025-08-08 06:43:33.813065,1
2,10001,PARAFUSO,BT10,100,100,z0019_1.csv,2025-08-08 06:24:36.622840,1
3,10002,MARTELO,BT56,100,1500,z0019_1.csv,2025-08-08 06:24:36.622840,1
4,10004,SERRA,BT56,100,200,z0019_2.csv,2025-08-08 06:43:33.813065,1


In [8]:
#aplicar o significado do glosario nas colunas
# Remove as colunas auxiliares que não serão mais utilizadas
# - nome_arquivo: origem dos dados (já não é necessário)
# - data_ingestao: usada para ordenação
# - row: gerada pela função ROW_NUMBER() para selecionar os registros mais recentes
df_final = df.drop(columns=['nome_arquivo', 'data_ingestao', 'row'])

# Exibe os 10 primeiros registros resultantes após a limpeza
df_final.head(10)


,NATBR,MATKT,WERKS,MAINS,LABST
0,10003,PREGO,BT10,100,60
1,10005,MACHADO,BT56,100,100
2,10001,PARAFUSO,BT10,100,100
3,10002,MARTELO,BT56,100,1500
4,10004,SERRA,BT56,100,200


In [18]:
df_final = df_final.rename(columns={"NATBR": "id"})
df_final = df_final.rename(columns={"MATKT": "nome"})
df_final = df_final.rename(columns={"WERKS": "categoria"})
df_final = df_final.rename(columns={"MAINS": "fornecedor"})
df_final = df_final.rename(columns={"LABST": "preco"})
df_final.head(10)


,id,nome,categoria,fornecedor,preco
0,10003,PREGO,BT10,100,60
1,10005,MACHADO,BT56,100,100
2,10001,PARAFUSO,BT10,100,100
3,10002,MARTELO,BT56,100,1500
4,10004,SERRA,BT56,100,200


In [ ]:
# TIPO ESTA TUDO COMO OBJETO, TEM QUE MUDAR ISSO
df_final.dtypes

id            object
nome          object
categoria     object
fornecedor    object
preco         object
dtype: object

In [23]:
# Cria df2 a partir do df_final
df2 = df_final.copy()

# Converte os tipos de cada coluna
df2 = df2.astype({
    "id": "int32",           # inteiro
    "nome": "string",        # texto
    "categoria": "string",   # texto
    "fornecedor": "int32",  # texto
    "preco": "float32"       # número decimal
})

# Visualiza os primeiros registros para conferir
df2.head(10)


,id,nome,categoria,fornecedor,preco
0,10003,PREGO,BT10,100,60.0
1,10005,MACHADO,BT56,100,100.0
2,10001,PARAFUSO,BT10,100,100.0
3,10002,MARTELO,BT56,100,1500.0
4,10004,SERRA,BT56,100,200.0


In [24]:
# TIPO ESTA OK AGORA
df2.dtypes

id                     int32
nome          string[python]
categoria     string[python]
fornecedor             int32
preco                float32
dtype: object

In [27]:
# df_final já está com os nomes finais: id, nome, categoria, fornecedor, preco
df2 = df_final.copy()

df2 = df2.astype({
    'id': int,           # id do produto
    'nome': str,         # nome/descrição
    'categoria': str,    # categoria
    'fornecedor': int,   # fornecedor
    'preco': float       # preço
})

# conferir
df2.dtypes
# df2.head(10)


id              int64
nome           object
categoria      object
fornecedor      int64
preco         float64
dtype: object

In [28]:
con.execute("""
CREATE TABLE IF NOT EXISTS produtos (
    id BIGINT,
    nm_produto TEXT,
    id_categoria TEXT,
    id_fornecedor BIGINT,
    vl_preco FLOAT
)
""")


In [ ]:
#inserir o df2 na nova tabela produtos em seguida
df2.head(10)

,id,nome,categoria,fornecedor,preco
0,10003,PREGO,BT10,100,60.0
1,10005,MACHADO,BT56,100,100.0
2,10001,PARAFUSO,BT10,100,100.0
3,10002,MARTELO,BT56,100,1500.0
4,10004,SERRA,BT56,100,200.0


In [30]:
df_resultado = con.execute("SELECT * FROM produtos").fetchdf()
df_resultado.head(10)


,id,nm_produto,id_categoria,id_fornecedor,vl_preco


In [31]:
con.execute("INSERT INTO produtos SELECT * FROM df2")


In [ ]:
#agora a tabela produtos esta com dados prontos
df_resultado = con.execute("SELECT * FROM produtos").fetchdf()
df_resultado.head(10)


,id,nm_produto,id_categoria,id_fornecedor,vl_preco
0,10003,PREGO,BT10,100,60.0
1,10005,MACHADO,BT56,100,100.0
2,10001,PARAFUSO,BT10,100,100.0
3,10002,MARTELO,BT56,100,1500.0
4,10004,SERRA,BT56,100,200.0


In [33]:
con.close()